In [1]:
import pandas as pd

In [2]:
scores = pd.read_csv("NFLPreseason.csv")
stats = pd.read_csv("NFLPreseasonTurnDiff.csv")


games = scores.home_team
teams = stats.team

stats.head()

,team,turnDiff,dypp,oypg,nypg
0,BAL,1,3.8,4.8,1.0
1,NE,0,3.6,5.4,1.8
2,NYG,0,4.4,6.2,1.8
3,ATL,0,4.4,4.3,-0.1
4,DEN,-2,4.7,3.7,-1.0


In [3]:
scores.head()

,away_team,home_team,total_away_score,total_home_score
0,DEN,ATL,14,10
1,NYJ,NYG,22,31
2,IND,BUF,16,24
3,ATL,MIA,27,34
4,NE,DET,31,3


In [4]:
columns = ['team', 'rpi', 'pyth_wins', 'turn_diff', 'dypp', 'oypp', 'nypp', 'wp', 'oppr', 'oor']
df = pd.DataFrame(columns=columns)
df['team'] = teams
df['turn_diff'] = stats.turnDiff
df['dypp'] = stats.dypp
df['oypp'] = stats.oypg
df['nypp'] = stats.nypg

In [5]:
pfs = []
pas = []
ws = []
ls = []
for team in teams:
    pf = scores[(scores['home_team']==team)].total_home_score.sum()
    pa = scores[(scores['home_team']==team)].total_away_score.sum()
    pa += scores[(scores['away_team']==team)].total_home_score.sum()
    pf += scores[(scores['away_team']==team)].total_away_score.sum()
    
    count = scores[(scores['home_team']==team)].home_team.count() + scores[(scores['away_team']==team)].home_team.count()
    pf /= count
    pa /= count
    
    wins = scores[(scores['home_team']==team) & (scores['total_home_score'] > scores['total_away_score'])].home_team.count()
    losses = scores[(scores['home_team']==team) & (scores['total_home_score'] < scores['total_away_score'])].home_team.count()
    wins += scores[(scores['away_team']==team) & (scores['total_home_score'] < scores['total_away_score'])].home_team.count()
    losses += scores[(scores['away_team']==team) & (scores['total_home_score'] > scores['total_away_score'])].home_team.count()
    
    pfs.append(pf)
    pas.append(pa)
    ws.append(wins)
    ls.append(losses)
    
    df['pf'] = pd.Series(pfs)
    df['pa'] = pd.Series(pas)
    df['wins'] = pd.Series(ws)
    df['losses'] = pd.Series(ls)
  

In [6]:
wps = []
opprs = []
oors = []
rpis = []
pythWinss = []

def calcOOR(opp):
    opps = scores[scores.home_team == opp].away_team.tolist() + scores[scores.away_team == opp].home_team.tolist()
    wp3 = 0
    for team in opps:
        wp3 += (df[df.team == team].wins.sum())/(df[df.team == team].wins.sum()+df[df.team == team].losses.sum())/len(opps)
    return wp3
def calcOR(team, opp, opps):
    oppWP = 0
    wp2 = (df[df.team == opp].wins.sum())/(df[df.team == opp].wins.sum()+df[df.team == opp].losses.sum())
    for opp in opps:
        oppWP += (calcOOR(opp))/len(opps)
    #print("oor1", oppWP)
    oors.append(oppWP)
    return wp2
 
def calcWP(team):
    opps = scores[scores.home_team == team].away_team.tolist() + scores[scores.away_team == team].home_team.tolist()
    #print('opps', opps)
    oppWP = 0
    wp = (df[df.team == team].wins.sum())/(df[df.team == team].wins.sum()+df[df.team == team].losses.sum())
    for opp in opps:
        oppWP += (calcOR(team, opp, opps))/len(opps)
    #print(team, 'wp', wp, 'or', oppWP)
    wps.append(wp)
    opprs.append(oppWP)
    
def calcRPI(team):
    rpi = (0.25 * df[df.team == team].wp.sum()) + (0.5 * df[df.team == team].oppr.sum()) + (0.25 * df[df.team == team].oor.sum())
    #print(rpi)
    rpis.append(rpi)

def calcPythWins(team):
    pythWins = (df[df.team == team].pf.sum()**2.37) / (df[df.team == team].pf.sum()**2.37 + df[df.team == team].pa.sum()**2.37)
    #print(pythWins)
    pythWinss.append(pythWins)


for team in teams:
    calcWP(team)
    calcPythWins(team)
    
df['wp'] = pd.Series(wps)
df['oppr'] = pd.Series(opprs)
df['oor'] = pd.Series(oors)
df['pyth_wins'] = pd.Series(pythWinss)

for team in teams:
    calcRPI(team)

df['rpi'] = pd.Series(rpis)
    
df


,team,rpi,pyth_wins,turn_diff,dypp,oypp,nypp,wp,oppr,oor,pf,pa,wins,losses
0,BAL,0.518519,0.925361,1,3.8,4.8,1.0,1.000000,0.222222,0.629630,27.000000,9.333333,3,0
1,NE,0.518519,0.915916,0,3.6,5.4,1.8,1.000000,0.222222,0.629630,21.000000,7.666667,3,0
2,NYG,0.574074,0.728700,0,4.4,6.2,1.8,1.000000,0.333333,0.629630,29.333333,19.333333,3,0
3,ATL,0.392361,0.234302,0,4.4,4.3,-0.1,0.000000,0.395833,0.777778,13.500000,22.250000,0,4
4,DEN,0.506944,0.330513,-2,4.7,3.7,-1.0,0.250000,0.500000,0.777778,12.250000,16.500000,1,3
5,OAK,0.611111,0.682078,-2,4.8,5.3,0.5,1.000000,0.333333,0.777778,23.000000,16.666667,3,0
6,BUF,0.435185,0.723312,-2,4.4,5.8,1.4,1.000000,0.111111,0.518519,25.000000,16.666667,3,0
7,LAC,0.407407,0.344801,0,5.1,5.5,0.4,0.000000,0.555556,0.518519,15.000000,19.666667,0,3
8,SF,0.587963,0.768359,-1,4.0,5.5,1.5,1.000000,0.416667,0.518519,22.666667,13.666667,3,0
9,TEN,0.572917,0.500000,3,5.5,4.7,-0.8,0.333333,0.777778,0.402778,16.666667,16.666667,1,2


In [7]:
df2 = df[['team', 'rpi', 'pyth_wins', 'nypp']]
df2.rename(columns={'team':'home_team'}, inplace=True)
new_scores = pd.merge(scores, df2, on="home_team")

/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [8]:
#df = df[['team', 'rpi', 'pyth_wins', 'nypp']]
df2.rename(columns={'home_team':'away_team', 'rpi':'away_rpi', 'pyth_wins': 'away_pyth_wins', 'nypp':'away_nypp'}, inplace=True)
new_new_scores = pd.merge(new_scores, df2, on="away_team")


In [9]:
test = pd.read_csv('NFLPreseasonNew.csv')

In [10]:
test = pd.merge(test, df2, on="away_team")
df2.rename(columns={'away_team': 'home_team','away_rpi':'rpi', 'away_pyth_wins': 'pyth_wins', 'away_nypp':'nypp'}, inplace=True)
test = pd.merge(test, df2, on="home_team")

In [11]:
#df2.rename(columns={'home_team':'away_team', 'rpi':'away_rpi', 'pyth_wins': 'away_pyth_wins', 'nypp':'away_nypp'}, inplace=True)
#test = pd.merge(test, df2, on='away_team')


In [12]:
test

,away_team,home_team,away_rpi,away_pyth_wins,away_nypp,rpi,pyth_wins,nypp
0,IND,CIN,0.518519,0.306343,-0.2,0.477865,0.390643,-0.5
1,PHI,NYJ,0.479167,0.355348,0.2,0.513889,0.388695,-1.2
2,ATL,JAX,0.392361,0.234302,-0.1,0.450087,0.028812,-1.0
3,MIN,BUF,0.655093,0.720310,1.4,0.435185,0.723312,1.4
4,PIT,CAR,0.564815,0.748783,0.8,0.588976,0.370785,-1.3
5,BAL,WAS,0.518519,0.925361,1.0,0.402778,0.300416,-0.5
6,DET,CLE,0.481481,0.189202,-1.1,0.518519,0.734595,0.3
7,NYG,NE,0.574074,0.728700,1.8,0.518519,0.915916,1.8
8,KC,GB,0.572917,0.509633,0.3,0.625000,0.396678,-0.7
9,LAR,HOU,0.587963,0.283665,-0.2,0.423611,0.271110,-0.6


In [13]:
#df[(df.team == "NO") | (df.team == 'LAC')]

In [14]:
new_new_scores

,away_team,home_team,total_away_score,total_home_score,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp
0,DEN,ATL,14,10,0.392361,0.234302,-0.1,0.506944,0.330513,-1.0
1,DEN,SEA,14,22,0.475694,0.599327,-0.3,0.506944,0.330513,-1.0
2,DEN,LAR,6,10,0.587963,0.283665,-0.2,0.506944,0.330513,-1.0
3,NYJ,ATL,22,10,0.392361,0.234302,-0.1,0.513889,0.388695,-1.2
4,NYJ,NYG,22,31,0.574074,0.728700,1.8,0.513889,0.388695,-1.2
5,WAS,ATL,19,7,0.392361,0.234302,-0.1,0.402778,0.300416,-0.5
6,WAS,CLE,10,30,0.518519,0.734595,0.3,0.402778,0.300416,-0.5
7,CHI,NYG,13,32,0.574074,0.728700,1.8,0.432870,0.326050,-1.0
8,CHI,IND,27,17,0.518519,0.306343,-0.2,0.432870,0.326050,-1.0
9,IND,BUF,16,24,0.435185,0.723312,1.4,0.518519,0.306343,-0.2


In [15]:
import numpy as np
new_new_scores['home_win'] = np.where(new_new_scores.total_home_score > new_new_scores.total_away_score, 1, 0)

In [16]:
new_new_scores

,away_team,home_team,total_away_score,total_home_score,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,home_win
0,DEN,ATL,14,10,0.392361,0.234302,-0.1,0.506944,0.330513,-1.0,0
1,DEN,SEA,14,22,0.475694,0.599327,-0.3,0.506944,0.330513,-1.0,1
2,DEN,LAR,6,10,0.587963,0.283665,-0.2,0.506944,0.330513,-1.0,1
3,NYJ,ATL,22,10,0.392361,0.234302,-0.1,0.513889,0.388695,-1.2,0
4,NYJ,NYG,22,31,0.574074,0.728700,1.8,0.513889,0.388695,-1.2,1
5,WAS,ATL,19,7,0.392361,0.234302,-0.1,0.402778,0.300416,-0.5,0
6,WAS,CLE,10,30,0.518519,0.734595,0.3,0.402778,0.300416,-0.5,1
7,CHI,NYG,13,32,0.574074,0.728700,1.8,0.432870,0.326050,-1.0,1
8,CHI,IND,27,17,0.518519,0.306343,-0.2,0.432870,0.326050,-1.0,0
9,IND,BUF,16,24,0.435185,0.723312,1.4,0.518519,0.306343,-0.2,1


In [17]:
train = new_new_scores[['rpi', 'pyth_wins', 'nypp', 'away_rpi', 'away_pyth_wins', 'away_nypp']]

In [18]:
train_labels = list(new_new_scores['home_win'])

In [19]:
#pd.options.display.float_format = '{:.5f}'.format

In [20]:
a = train[['rpi', 'pyth_wins', 'nypp', 'away_rpi', 'away_pyth_wins', 'away_nypp']]


# import logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# specify the alphas
param_grid = {'C': np.logspace(-5, 5, 15)}

# Instantiate a logistic regression classifier: logreg
logit = LogisticRegression()

# specify five-fold cross validation logistic regression while tuning over alpha space
logit_cv = GridSearchCV(logit, param_grid, cv=5)

# fit the model
logit_cv.fit(a,train_labels)

/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: 

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([1.00000e-05, 5.17947e-05, 2.68270e-04, 1.38950e-03, 7.19686e-03,
       3.72759e-02, 1.93070e-01, 1.00000e+00, 5.17947e+00, 2.68270e+01,
       1.38950e+02, 7.19686e+02, 3.72759e+03, 1.93070e+04, 1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Compute predicted probabilities: y_pred_prob
a = train[['rpi', 'pyth_wins', 'nypp', 'away_rpi', 'away_pyth_wins', 'away_nypp']]
y_pred_prob = logit_cv.predict_proba(a)[:,1]

# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(train_labels, y_pred_prob)

#compute area under the curve
auc = auc(fpr,tpr)

# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label="AUC = %0.2f" % auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

<Figure size 640x480 with 1 Axes>

In [22]:
# predict the results 
a = test[['rpi', 'pyth_wins', 'nypp', 'away_rpi', 'away_pyth_wins', 'away_nypp']]
new_preds=logit_cv.predict(a)
new_new_test = a
new_new_test['win_prob'] = logit_cv.predict_proba(a)[:,1]
new_new_test['prediction'] = new_preds
#new_new_test['winner'] = list(test_labels)
#future_games[['home_team','away_team','winner','win_prob']]
new_new_test

/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction
0,0.477865,0.390643,-0.5,0.518519,0.306343,-0.2,0.629475,1
1,0.513889,0.388695,-1.2,0.479167,0.355348,0.2,0.390179,0
2,0.450087,0.028812,-1.0,0.392361,0.234302,-0.1,0.031545,0
3,0.435185,0.723312,1.4,0.655093,0.720310,1.4,0.762449,1
4,0.588976,0.370785,-1.3,0.564815,0.748783,0.8,0.007268,0
5,0.402778,0.300416,-0.5,0.518519,0.925361,1.0,0.000109,0
6,0.518519,0.734595,0.3,0.481481,0.189202,-1.1,0.999613,1
7,0.518519,0.915916,1.8,0.574074,0.728700,1.8,0.995842,1
8,0.625000,0.396678,-0.7,0.572917,0.509633,0.3,0.420581,0
9,0.423611,0.271110,-0.6,0.587963,0.283665,-0.2,0.142293,0


In [23]:
# predict the results 
a = test[['rpi', 'pyth_wins', 'nypp', 'away_rpi', 'away_pyth_wins', 'away_nypp']]
new_preds=logit_cv.predict(a)
new_new_test = a
new_new_test['win_prob'] = logit_cv.predict_proba(a)[:,1]
new_new_test['prediction'] = new_preds
#new_new_test['winner'] = list(test_labels)
#future_games[['home_team','away_team','winner','win_prob']]
new_new_test

/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction
0,0.477865,0.390643,-0.5,0.518519,0.306343,-0.2,0.629475,1
1,0.513889,0.388695,-1.2,0.479167,0.355348,0.2,0.390179,0
2,0.450087,0.028812,-1.0,0.392361,0.234302,-0.1,0.031545,0
3,0.435185,0.723312,1.4,0.655093,0.720310,1.4,0.762449,1
4,0.588976,0.370785,-1.3,0.564815,0.748783,0.8,0.007268,0
5,0.402778,0.300416,-0.5,0.518519,0.925361,1.0,0.000109,0
6,0.518519,0.734595,0.3,0.481481,0.189202,-1.1,0.999613,1
7,0.518519,0.915916,1.8,0.574074,0.728700,1.8,0.995842,1
8,0.625000,0.396678,-0.7,0.572917,0.509633,0.3,0.420581,0
9,0.423611,0.271110,-0.6,0.587963,0.283665,-0.2,0.142293,0


In [24]:
new_new_test['home_team'] = test['home_team']
new_new_test['away_team'] = test['away_team']
new_new_test

,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction,home_team,away_team
0,0.477865,0.390643,-0.5,0.518519,0.306343,-0.2,0.629475,1,CIN,IND
1,0.513889,0.388695,-1.2,0.479167,0.355348,0.2,0.390179,0,NYJ,PHI
2,0.450087,0.028812,-1.0,0.392361,0.234302,-0.1,0.031545,0,JAX,ATL
3,0.435185,0.723312,1.4,0.655093,0.720310,1.4,0.762449,1,BUF,MIN
4,0.588976,0.370785,-1.3,0.564815,0.748783,0.8,0.007268,0,CAR,PIT
5,0.402778,0.300416,-0.5,0.518519,0.925361,1.0,0.000109,0,WAS,BAL
6,0.518519,0.734595,0.3,0.481481,0.189202,-1.1,0.999613,1,CLE,DET
7,0.518519,0.915916,1.8,0.574074,0.728700,1.8,0.995842,1,NE,NYG
8,0.625000,0.396678,-0.7,0.572917,0.509633,0.3,0.420581,0,GB,KC
9,0.423611,0.271110,-0.6,0.587963,0.283665,-0.2,0.142293,0,HOU,LAR


In [25]:
picks = new_new_test[(new_new_test.win_prob >= 0.85) | (new_new_test.win_prob <= 0.15)]

In [26]:
picks.win_prob = picks.win_prob.apply(lambda x: '%.6f' % x)

/Users/samuelburgess/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [27]:
picks

,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction,home_team,away_team
2,0.450087,0.028812,-1.0,0.392361,0.234302,-0.1,0.031545,0,JAX,ATL
4,0.588976,0.370785,-1.3,0.564815,0.748783,0.8,0.007268,0,CAR,PIT
5,0.402778,0.300416,-0.5,0.518519,0.925361,1.0,0.000109,0,WAS,BAL
6,0.518519,0.734595,0.3,0.481481,0.189202,-1.1,0.999613,1,CLE,DET
7,0.518519,0.915916,1.8,0.574074,0.728700,1.8,0.995842,1,NE,NYG
9,0.423611,0.271110,-0.6,0.587963,0.283665,-0.2,0.142293,0,HOU,LAR
10,0.432870,0.326050,-1.0,0.572917,0.500000,-0.8,0.013322,0,CHI,TEN
11,0.545139,0.854569,0.3,0.672309,0.510485,0.4,0.993380,1,DAL,TB
12,0.481481,0.569337,-0.7,0.451389,0.689426,0.1,0.112902,0,NO,MIA
15,0.587963,0.768359,1.5,0.407407,0.344801,0.4,0.999931,1,SF,LAC


In [28]:
#from sklearn.metrics import accuracy_score
#print(accuracy_score(new_new_test['winner'], new_new_test['prediction']))

In [29]:
picks[picks.home_team == 'WAS']

,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction,home_team,away_team
5,0.402778,0.300416,-0.5,0.518519,0.925361,1.0,0.000109,0,WAS,BAL


In [31]:
picks[picks.home_team == 'CAR']

,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction,home_team,away_team
4,0.588976,0.370785,-1.3,0.564815,0.748783,0.8,0.007268,0,CAR,PIT


In [32]:
picks[picks.home_team == 'CLE']

,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction,home_team,away_team
6,0.518519,0.734595,0.3,0.481481,0.189202,-1.1,0.999613,1,CLE,DET


In [33]:
picks[picks.home_team == 'DAL']

,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction,home_team,away_team
11,0.545139,0.854569,0.3,0.672309,0.510485,0.4,0.993380,1,DAL,TB


In [35]:
picks[picks.home_team == 'SF']

,rpi,pyth_wins,nypp,away_rpi,away_pyth_wins,away_nypp,win_prob,prediction,home_team,away_team
15,0.587963,0.768359,1.5,0.407407,0.344801,0.4,0.999931,1,SF,LAC


In [36]:
df

,team,rpi,pyth_wins,turn_diff,dypp,oypp,nypp,wp,oppr,oor,pf,pa,wins,losses
0,BAL,0.518519,0.925361,1,3.8,4.8,1.0,1.000000,0.222222,0.629630,27.000000,9.333333,3,0
1,NE,0.518519,0.915916,0,3.6,5.4,1.8,1.000000,0.222222,0.629630,21.000000,7.666667,3,0
2,NYG,0.574074,0.728700,0,4.4,6.2,1.8,1.000000,0.333333,0.629630,29.333333,19.333333,3,0
3,ATL,0.392361,0.234302,0,4.4,4.3,-0.1,0.000000,0.395833,0.777778,13.500000,22.250000,0,4
4,DEN,0.506944,0.330513,-2,4.7,3.7,-1.0,0.250000,0.500000,0.777778,12.250000,16.500000,1,3
5,OAK,0.611111,0.682078,-2,4.8,5.3,0.5,1.000000,0.333333,0.777778,23.000000,16.666667,3,0
6,BUF,0.435185,0.723312,-2,4.4,5.8,1.4,1.000000,0.111111,0.518519,25.000000,16.666667,3,0
7,LAC,0.407407,0.344801,0,5.1,5.5,0.4,0.000000,0.555556,0.518519,15.000000,19.666667,0,3
8,SF,0.587963,0.768359,-1,4.0,5.5,1.5,1.000000,0.416667,0.518519,22.666667,13.666667,3,0
9,TEN,0.572917,0.500000,3,5.5,4.7,-0.8,0.333333,0.777778,0.402778,16.666667,16.666667,1,2
